In [497]:
import scipy as sc
import numpy as np
#!pip install gpxpy
import gpxpy
from geopy import distance
import json
import pandas as pd
import math
!pip install fit2gpx
from fit2gpx import Converter

# Putting it all together:

1. read gpx file(s) into an array with elevations and coordinates
2. read segement data into array with elevations and coordinates
3. get total distance and total elevation difference from segment
4. sweep through gpx arrays to find sections that roughly fit the segment
5. calculate elapsed times for those sections
6. use DTW to see if segments are actually similar
    - if they are, compare times.

- Thought - skip the prelim data sweep and just use DTW

In [498]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)])
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            #print(cost)
            # take last min from a square box
            prev_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + prev_min
    return dtw_matrix

In [499]:
def dtw(s, t):
    n, m = len(s), len(t)
    dtw_matrix = np.zeros((n+1, m+1))
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            prev_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + prev_min
    return dtw_matrix

In [500]:
from geopy import distance
import math
# method to calculate distances between point
def distanceBetweenPoints(p1, p2):
    flat_distance = distance.distance(p1[:2], p2[:2]).m
    # print(flat_distance)
    euclidian_distance = math.sqrt(flat_distance**2 + (p2[2] - p1[2])**2)
    return euclidian_distance

In [501]:
from geopy import Point
import math
# method to make GPX file into array of elevations, times, distances
def file_to_elevation_arr(fileName):
        gpx_file = open(fileName, 'r')
        gpx = gpxpy.parse(gpx_file)
        elevevation_time_and_distance = []
        pointOne = None
        for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    pointTwo = point
                    distance = 0
                    if pointOne:
                        p1 = Point([pointOne.latitude, pointOne.longitude, pointOne.elevation])
                        p2 = Point([pointTwo.latitude, pointTwo.longitude, pointTwo.elevation])
                        distance = distanceBetweenPoints(p1, p2)
                    pointOne = point
                    #print('Point at ({0},{1}) -> {2}'.format(point.latitude, point.longitude, point.elevation))
                    elevevation_time_and_distance.append((point.elevation,point.time, distance))
        return elevevation_time_and_distance

In [502]:
fileOne = "data/Walt's Strava/activities/5378707142.gpx"
fileTwo  = "data/Walt's Strava/activities/4812864781.gpx"

In [503]:
arrOne = file_to_elevation_arr(fileOne)
arrTwo = file_to_elevation_arr(fileTwo)

In [504]:
segment = "data/Elevation JSON/segment_location_data_elevation_0.json"

In [505]:
def dataframe_segment_to_list(df):
    sg =[]
    # Iterate over each row
    for index, rows in df.iterrows():
        # Create list for the current row
        my_list =[rows[2], rows[3]]
        #print(my_list)
        
        # append the list to the final list
        sg.append(my_list)
    return sg
  

In [506]:
#print(segment_data)
def total_segment_elevation(segment_data):
    total_elev = segment_data[len(segment_data)-1][0] - segment_data[0][0]
    return total_elev

def total_segment_distance(segment_data):
    total_distance = 0
    for i in segment_data:
        total_distance += i[1].item()
    return total_distance

In [507]:
def sweep_user_activity_old(user_data, segement_data, x):
    list_of_matches = [] # list of lists

    seg_distance = total_segment_distance(segement_data)

    #print(seg_distance)
    seg_elevation = total_segment_elevation(segement_data)
    #print(seg_elevation)
    
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        # until we get to the window 
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window += 1
    delta_distance = user_distance - user_data[0][2]
    # now we have the window, so we need to iterate through and see if the profiles match
    second_window = 0
    second_distance = 0
    user_elev_two = user_data[0][0]
    while user_distance < user_data[-1][2] and first_window < len(user_data):
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        if user_elev_two - user_elev > seg_elevation-5 and  user_elev_two - user_elev < seg_elevation+5:
            list_of_matches.append(user_data[second_window:first_window])
            #print(first_window, second_window)
            # add match to list
        user_distance += user_data[first_window][2]
        second_distance += user_data[second_window][2]
        first_window +=1
        second_window +=1
    return list_of_matches




In [508]:
# What does this need to do?
# Go to segment distance, when the segment distance matches the user window, compare elevations
def sweep_user_activity(user_data, segement_data, window):
    list_of_matches = [] # list of lists
    seg_distance = total_segment_distance(segement_data) # total segment distance
    seg_elevation = total_segment_elevation(segement_data) # total segment elevation
    
    # go to the correct distance in user
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window+=1
    second_window = 0 
    second_distance = 0
    while first_window < len(user_data)-1: 
        while user_distance-second_distance < seg_distance and first_window < len(user_data)-1:
            user_distance += user_data[first_window][2]
            first_window+=1
        user_elev_two = user_data[first_window][0]
        user_elev = user_data[second_window][0]
        second_distance += user_data[second_window][2]
        if first_window % 100 == 0:
            if user_elev_two - user_elev > seg_elevation-window and  user_elev_two - user_elev < seg_elevation+window:
                list_of_matches.append(user_data[second_window:first_window])
        second_window+= 1
    return list_of_matches
        


In [509]:
def sweep_user_activity_old(user_data, segement_data, window):
    list_of_matches = [] # list of lists
    seg_distance = total_segment_distance(segement_data) # total segment distance
    seg_elevation = total_segment_elevation(segement_data) # total segment elevation
    
    # go to the correct distance in user
    user_distance = 0
    user_elev = user_data[0][0]
    user_start_elev = user_data[0][0]
    first_window = 0
    while user_distance < seg_distance and first_window < len(user_data):
        user_elev = user_data[first_window][0]-user_start_elev
        user_distance += user_data[first_window][2]
        first_window+=1
    

In [510]:
def read_segment(segmentFile):
    segment = segmentFile
    df = pd.read_json(segment)
    df.head()
    seg = dataframe_segment_to_list(df)
    return seg

In [511]:
claremont_turkey_trot = read_segment('data/Elevation JSON/segment_location_data_elevation_0.json')


In [512]:
#print(arr[0])

In [513]:
import matplotlib.pyplot as plt
    

In [514]:
import numpy as np
import matplotlib.pyplot as plt
def graphing(user, segment):
    total_user_distance = 0
    total_segment_distance1 = 0
    user_start_elev = user[0][0]
    segment_start_elev = segment[0][0]
    user_dtw_list = []
    segment_dtw_list = []
    user_dtw_distance = []
    seg_dtw_distance = []
    for i in range(len(user)):
        total_user_distance += user[i][2]
        user_dtw_distance.append(total_user_distance)
        user_dtw_list.append(user[i][0]-user_start_elev)
        #plt.scatter(total_user_distance, user[i][0])
        #print(arr[0][i][0])
    for j in range (len(segment)):
         total_segment_distance1 += segment[j][1]
         seg_dtw_distance.append(total_segment_distance1)
         segment_dtw_list.append(segment[j][0]-segment_start_elev)
         #plt.scatter(total_segment_distance1, segment[j][0])
    #plt.draw()
    return ((user_dtw_list,user_dtw_distance), (segment_dtw_list,seg_dtw_distance))

In [515]:
from scipy.interpolate import interp1d
import numpy as np
# this is to correct the length of the segment for use in better DTW algorithms, like 
# the ones I have imported. My implementation does mot require this. 
def correct_segment(l):
    #l = graphing(arr[0], claremont_turkey_trot) #(elev, distance), (elev, distance)
    #print(l[0][1])
    #plt.plot(l[0][1],l[0][0],'o')
    f = interp1d(l[1][1],l[1][0], kind='cubic')
    segment_corrected_length = []
    desired_length = len(l[0][0])
    divided = l[1][1][-1]//desired_length
    for i in range(desired_length-1):
        segment_corrected_length.append(f(i*divided))
    #plt.plot(l[1][1], f(l[1][1]), '-')
    #plt.show()
    return segment_corrected_length


In [516]:
from scipy.interpolate import interp1d
import numpy as np
#!pip install dtaidistance
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#l = graphing(arr[0], claremont_turkey_trot)
def DTW_comparison(user, segment):
    distances = []
    user_times = []
    d_t = []
    for pot in range(len(user)-1):
        t1 = user[pot][0][1]
        t2 = user[pot][-1][1]
        #print(t1)
        user_times.append((t2-t1).total_seconds())
        l = graphing(user[pot], segment) #(elev, distance), (elev, distance)
        first = l[0][0]
        second = l[1][0]
        #f2 = interp1d(l[0][0], l[0][1], kind='cubic')
        tempDTW = dtw(first, second)
        distances.append(tempDTW[-1][-1])
        #distances.append(dtw.distance(first, second))

    
    for d in range(len(distances)):
        d_t.append((distances[d],user_times[d]))
    return d_t
    


[(20697.829999999834, 1682.0), (20697.829999999834, 1681.0)]

In [517]:
#DTW_comparison(arr, claremont_turkey_trot)

In [518]:
#!pip install dtaidistance
#from dtaidistance import dtw
#from dtaidistance import dtw_visualisation as dtwvis
#s = correct_segment(l)
#path = dtw.warping_path(l[0][0], s)
#dtwvis.plot_warping(l[0][0], s, path, filename="warp.png")
#distance = dtw.distance(l[0][0], s)

In [519]:
conv = Converter() 
def fit_to_gpx(fileDirectory):
    conv.fit_to_gpx_bulk(dir_in=fileDirectory, dir_out='data/User')



In [520]:
#fit_to_gpx('data/james_activities')

In [523]:
import os
def compare_user_activities_to_segment(segmentName):
    # scrape for segment
    data = []
    seg = read_segment(segmentName)
    # go through user's activities
    directory = os.fsencode('/Users/wjones/School/DSA/HunterXKOM/data/Mix_data')    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".gpx"): 
            arrOne = file_to_elevation_arr('/Users/wjones/School/DSA/HunterXKOM/data/Mix_data/'+filename)
            arr = sweep_user_activity(arrOne, seg, 5)
            data += DTW_comparison(arr, seg)
        else:
            continue
    return data

In [524]:
compare_user_activities_to_segment('data/Elevation JSON/segment_location_data_elevation_0.json')

[(1558.5400000000052, 1378.0),
 (2373.210000000043, 1719.0),
 (4469.0300000000125, 1682.0),
 (2601.7699999999927, 1650.0),
 (2620.7100000000023, 1613.0),
 (2919.589999999998, 1590.0),
 (2839.3200000000134, 1579.0),
 (2882.2500000000136, 1061.0),
 (3211.1799999999953, 1062.0),
 (3249.1099999999938, 1061.0),
 (2494.5899999999983, 1061.0),
 (2498.4700000000116, 1061.0),
 (2665.029999999992, 1061.0),
 (2936.779999999985, 1062.0),
 (2649.20000000003, 1077.0),
 (2739.149999999981, 1095.0),
 (2879.200000000007, 1093.0),
 (2879.200000000007, 1092.0),
 (2656.58000000001, 1068.0),
 (2646.6600000000058, 1102.0),
 (2821.209999999996, 1128.0),
 (2495.5199999999863, 1151.0)]